In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

def german_dictionary(user_input):
    session = requests.session()
    base_site = f"https://www.duden.de/rechtschreibung/{user_input.lower()}"
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"}
    response = session.get(base_site, headers=headers)
    if response.status_code == 200:
        try:
            html = response.content
            soup = BeautifulSoup(html, "lxml")
            # with open("duden.html", mode="wb") as file:
            #     file.write(soup.prettify("utf8"))
            second_site = "https://www.linguee.com/english-german/search"
            second_response = session.get(second_site, params= {"source":"german", "query":{user_input.lower()}}, headers=headers)
            second_html = second_response.content
            second_soup = BeautifulSoup(second_html, "lxml")
            session.close()
            try:
                article = soup.find("span", class_="lemma__determiner").text.replace("\xad","").replace(" oder",",")
            except AttributeError:
                article = "-"
            type_gender = soup.find("dd", class_="tuple__val").text.replace("\xad","")
            type_gender
            if "Substantiv" in type_gender or "substantiv" in type_gender:
                if len(type_gender.split(", ")) == 2:
                    typ, gender = type_gender.split(", ")
                    if gender.endswith("in"):
                        gender = (gender + "um").capitalize()
                    else:
                        gender = gender.capitalize()
                else:
                    type_gender = type_gender.replace(", oder",";")
                    gender = ""
                    for item in type_gender.split("; "):
                        _, gender1 = item.split(", ")
                        if gender1.endswith("in"):
                            gender1 = (gender1 + "um").capitalize()
                        else:
                            gender1 = gender1.capitalize()
                        gender += f"{gender1}, "
                    typ = type_gender.split("; ")[0].split(", ")[0]
                    gender = gender.strip(", ")
            elif "Verb" in type_gender:
                typ = "Verb"
                gender = "-"
            elif "\n" in type_gender:
                typ = "-"
                gender = "-"
            else:
                typ = type_gender
                gender = "-"
            try:
                synonym = soup.find("div", attrs={"id":"synonyme"}).find("li").text
            except AttributeError:
                synonym = "-"
            try:
                grammar = soup.find("div", id= "grammatik").find("p").text
                german = grammar[:grammar.find(";")].replace(" oder",",").replace(" auch:","")
                try:
                    if "der" not in grammar and "die" not in grammar and "das" not in grammar:
                        german = user_input.lower()
                        genitiv = "-"
                        plural = "-"
                    elif "und" not in grammar and "oder" not in grammar or "," in german:
                        genitiv, plural = grammar[grammar.find(";") + 2:].rsplit(", ", 1)
                        genitiv = genitiv.replace("Genitiv: ","")
                        plural = plural.replace("Plural: ","")
                        if plural.startswith("die") == False:
                            plural = f"die {plural}"
                    else:
                        grammar = grammar.replace(" und","?").replace(" oder","?")
                        # print(len(grammar.split("? ")[0][grammar.split("? ")[0].find(";") + 2:].replace(",","&",1).split("& ")))
                        genitiv = ""
                        plural = ""
                        for item in grammar.split("? "):
                            genitiv1, plural1 = item[item.find(";") + 2:].replace(",","&",1).split("& ")
                            genitiv1 = genitiv1.replace("Genitiv: ","")
                            plural1 = plural1.replace("Plural: ","")
                            genitiv += f"{genitiv1}, "
                            plural += f"{plural1}, "
                        german = f"{german[:3]}, {grammar.split('? ')[1][:4]}{german[4:]}"
                        genitiv = genitiv.strip(", ")
                        plural = plural.strip(", ")
                except ValueError:
                    if "?" not in grammar or "," in german:
                        genitiv = grammar[grammar.find(";") + 2:]
                        genitiv = genitiv.replace("Genitiv: ","")
                        plural = "-"
                    else:
                        genitiv = ""
                        for item in grammar.split("? "):
                            genitiv1 = item[item.find(";") + 2:]
                            genitiv1 = genitiv1.replace("Genitiv: ","")
                            genitiv += f"{genitiv1}, "
                        german = f"{german[:3]}, {grammar.split('? ')[1][:4]}{german[4:]}"
                        genitiv = genitiv.strip(", ")
                        plural = "-"
            except AttributeError:
                if typ == "Substantiv":
                    german = f"{article} {user_input.capitalize()}"
                else:
                    german = user_input.lower()
                genitiv = "-"
                plural = "-"
            try:
                information = second_soup.find("div", class_ = "lemma featured")
                english_information = information.find_all("a", class_ = "dictLink featured")
                english = [word.text.capitalize() for word in english_information]
                english_terms = ", ".join(english)
            except AttributeError:
                english_terms = "-"
            word_list = []
            word_list.append(german)
            word_list.append(typ)
            word_list.append(gender)
            word_list.append(article)
            word_list.append(genitiv)
            word_list.append(plural)
            word_list.append(synonym)
            word_list.append(english_terms)
            df = pd.DataFrame(word_list).transpose()
            df.columns=["Wort", "Typ", "Genus", "Artikel", "Genitiv", "Plural", "Synonym", "Englisch"]
            df.index = [""]
            # pd.set_option("display.max_colwidth", None)
            return df
        except IndexError:
            return "Word not found."
    else:
        base_site = f"https://www.duden.de/suchen/dudenonline/{user_input.lower()}"
        try:
            response = session.get(base_site, headers=headers)
            # response.status_code
            html = response.content
            soup = BeautifulSoup(html, "lxml")
            url = soup.find_all("a", class_= "vignette__link")[0]["href"]
            full_url = urljoin(base=base_site, url=url)
            response = session.get(full_url, headers=headers)
            # response.status_code
            html = response.content
            soup = BeautifulSoup(html, "lxml")
            # with open("duden.html", mode="wb") as file:
            #     file.write(soup.prettify("utf8"))
            second_site = "https://www.linguee.com/english-german/search"
            second_response = session.get(second_site, params= {"source":"german", "query":{user_input.lower()}}, headers=headers)
            second_html = second_response.content
            second_soup = BeautifulSoup(second_html, "lxml")
            session.close()
            try:
                article = soup.find("span", class_="lemma__determiner").text.replace("\xad","").replace(" oder",",")
            except AttributeError:
                article = "-"
            type_gender = soup.find("dd", class_="tuple__val").text.replace("\xad","")
            type_gender
            if "Substantiv" in type_gender or "substantiv" in type_gender:
                if len(type_gender.split(", ")) == 2:
                    typ, gender = type_gender.split(", ")
                    if gender.endswith("in"):
                        gender = (gender + "um").capitalize()
                    else:
                        gender = gender.capitalize()
                else:
                    type_gender = type_gender.replace(", oder",";")
                    gender = ""
                    for item in type_gender.split("; "):
                        _, gender1 = item.split(", ")
                        if gender1.endswith("in"):
                            gender1 = (gender1 + "um").capitalize()
                        else:
                            gender1 = gender1.capitalize()
                        gender += f"{gender1}, "
                    typ = type_gender.split("; ")[0].split(", ")[0]
                    gender = gender.strip(", ")
            elif "Verb" in type_gender:
                typ = "Verb"
                gender = "-"
            elif "\n" in type_gender:
                typ = "-"
                gender = "-"
            else:
                typ = type_gender
                gender = "-"
            try:
                synonym = soup.find("div", attrs={"id":"synonyme"}).find("li").text
            except AttributeError:
                synonym = "-"
            try:
                grammar = soup.find("div", id= "grammatik").find("p").text
                german = grammar[:grammar.find(";")].replace(" oder",",").replace(" auch:","")
                try:
                    if "der" not in grammar and "die" not in grammar and "das" not in grammar:
                        german = user_input.lower()
                        genitiv = "-"
                        plural = "-"
                    elif "und" not in grammar and "oder" not in grammar or "," in german:
                        genitiv, plural = grammar[grammar.find(";") + 2:].rsplit(", ", 1)
                        genitiv = genitiv.replace("Genitiv: ","")
                        plural = plural.replace("Plural: ","")
                        if plural.startswith("die") == False:
                            plural = f"die {plural}"
                    else:
                        grammar = grammar.replace(" und","?").replace(" oder","?")
                        # print(len(grammar.split("? ")[0][grammar.split("? ")[0].find(";") + 2:].replace(",","&",1).split("& ")))
                        genitiv = ""
                        plural = ""
                        for item in grammar.split("? "):
                            genitiv1, plural1 = item[item.find(";") + 2:].replace(",","&",1).split("& ")
                            genitiv1 = genitiv1.replace("Genitiv: ","")
                            plural1 = plural1.replace("Plural: ","")
                            genitiv += f"{genitiv1}, "
                            plural += f"{plural1}, "
                        german = f"{german[:3]}, {grammar.split('? ')[1][:4]}{german[4:]}"
                        genitiv = genitiv.strip(", ")
                        plural = plural.strip(", ")
                except ValueError:
                    if "?" not in grammar or "," in german:
                        genitiv = grammar[grammar.find(";") + 2:]
                        genitiv = genitiv.replace("Genitiv: ","")
                        plural = "-"
                    else:
                        genitiv = ""
                        for item in grammar.split("? "):
                            genitiv1 = item[item.find(";") + 2:]
                            genitiv1 = genitiv1.replace("Genitiv: ","")
                            genitiv += f"{genitiv1}, "
                        german = f"{german[:3]}, {grammar.split('? ')[1][:4]}{german[4:]}"
                        genitiv = genitiv.strip(", ")
                        plural = "-"
            except AttributeError:
                if typ == "Substantiv":
                    german = f"{article} {user_input.capitalize()}"
                else:
                    german = user_input.lower()
                genitiv = "-"
                plural = "-"
            try:
                information = second_soup.find("div", class_ = "lemma featured")
                english_information = information.find_all("a", class_ = "dictLink featured")
                english = [word.text.capitalize() for word in english_information]
                english_terms = ", ".join(english)
            except AttributeError:
                english_terms = "-"
            word_list = []
            word_list.append(german)
            word_list.append(typ)
            word_list.append(gender)
            word_list.append(article)
            word_list.append(genitiv)
            word_list.append(plural)
            word_list.append(synonym)
            word_list.append(english_terms)
            df = pd.DataFrame(word_list).transpose()
            df.columns=["Wort", "Typ", "Genus", "Artikel", "Genitiv", "Plural", "Synonym", "Englisch"]
            df.index = [""]
            pd.set_option("display.max_colwidth", None)
            return df
        except IndexError:
            return "Word not found."

In [2]:
word = input("Please enter the word: ")
german_dictionary(word)

,Wort,Typ,Genus,Artikel,Genitiv,Plural,Synonym,Englisch
,die Benachrichtigung,Substantiv,Femininum,die,der Benachrichtigung,die Benachrichtigungen,"Information, Informierung, Unterrichtung, Verständigung","Notification, Notice, Information"
